In [1]:
import pandas as pd
import time
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from utils_processor.processor import Processor
from nltk.corpus import stopwords
import pandas as pd
import nltk
import sklearn
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, SnowballStemmer
from utils_processor.processor import Processor
import re 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np



In [2]:
processor_ = Processor()

In [3]:
def print_progress_bar(percentage):
    bar_length = 50  # Definir la longitud de la barra de progreso
    filled_length = int(bar_length * percentage // 100)
    bar = '█' * filled_length + '-' * (bar_length - filled_length)
    print(f'\rProgress: |{bar}| {percentage}% Complete', end='\r')

def log_category_progress(category, step, total_steps):
    percentage = int((step / total_steps) * 100)
    print(f"\nProcessing {category} ({step}/{total_steps})")
    print_progress_bar(percentage)
    time.sleep(0.5)  # Solo para simular el tiempo de procesamiento

def read_file(file_location) -> pd.DataFrame:
    print(f"Reading file: {file_location}")
    with open(file_location, 'r') as file:
        data = file.readlines()

    rows = []
    for i in range(0, len(data)):
        terms = {}
        # Separar la parte de los términos del label
        line_parts = data[i].split(" #label#:")
        terms_part = line_parts[0].split()  # Parte con los términos
        label = line_parts[1].strip()  # El label (positive o negative)

        # Iterar sobre los términos y extraer el valor
        for j in terms_part:
            if ":" in j:
                term, value = j.split(":")
                terms[term] = int(value)

        # Crear un diccionario para la fila con el formato { 'terms': {términos}, 'label': label }
        rows.append({
            "terms": terms,  # El diccionario con los términos y sus valores
            "label": label   # El label ('positive' o 'negative')
        })

    # Crear un DataFrame con dos columnas: 'terms' y 'label'
    dataFrame = pd.DataFrame(rows)

    return dataFrame

In [4]:
df_train = read_file("data/MultiDomainSentiment/negative.review")
df_train

Reading file: data/MultiDomainSentiment/negative.review


,terms,label
0,"{'avid': 1, 'your': 1, 'horrible_book': 1, 'wa...",negative
1,"{'to_use': 1, 'shallow': 1, 'found': 1, 'he_ca...",negative
2,"{'avid': 1, 'your': 1, 'horrible_book': 1, 'wa...",negative
3,"{'book_seriously': 1, 'we': 1, 'days_couldn't'...",negative
4,"{'mass': 1, 'only': 1, 'he': 2, 'help': 1, '""j...",negative
...,...,...
995,"{'only': 1, 'idiotic_anyone': 1, 'if_i': 1, 'm...",negative
996,"{'your': 1, 'well': 1, 'to_create': 1, 'peter'...",negative
997,"{'favorable_reviews': 1, 'heard': 1, 'straight...",negative
998,"{'helpful': 1, 'this_one': 1, 'substance_and':...",negative


In [5]:
df_train.to_csv("negative.review.csv")

In [6]:
df_validation = read_file("data/MultiDomainSentiment/unlabeled.review")
df_validation

Reading file: data/MultiDomainSentiment/unlabeled.review


,terms,label
0,"{'is_such': 1, 'feel': 1, 'pages': 1, 'if': 1,...",negative
1,"{'go_mercy': 1, 'forget': 1, 'all_the': 1, 'pi...",negative
2,"{'explanation_of': 1, 'plains': 1, 'bison': 5,...",positive
3,"{'stars': 1, 'bold_kudos': 1, 'every_pedophile...",positive
4,"{'doesn't_say': 1, 'their_class': 1, 'say_much...",negative
...,...,...
4460,"{'mass': 1, 'specifically_references': 1, 'sci...",negative
4461,"{'reviewer's_comments': 1, 'to_pick': 1, 'comm...",negative
4462,"{'x-ers': 1, 'entry-level': 1, 'can_dip': 1, '...",positive
4463,"{'your': 1, 'well': 1, 'around_for': 1, 'you'r...",positive


In [7]:
df_validation.to_csv("unlabeled.review.csv")

In [8]:
def process_category(category, negative_file, positive_file, testing_file):
    total_steps = 3
    step = 1

    log_category_progress(category, step, total_steps)
    df_negative = read_file(negative_file)

    step += 1
    log_category_progress(category, step, total_steps)
    df_positive = read_file(positive_file)

    step += 1
    log_category_progress(category, step, total_steps)
    df_testing = read_file(testing_file)

    # Combinar los datasets de entrenamiento y testing
    df_train = pd.concat([df_negative, df_positive])
    df_train["category"] = category
    df_testing["category"] = category

    return df_train, df_testing

In [9]:
# Procesar cada categoría

print("Starting processing...")

# Kitchen
df_kitchen, df_kitchen_testing = process_category(
    "kitchen",
    "data/MultiDomainSentiment/processed_acl/kitchen/negative.review",
    "data/MultiDomainSentiment/processed_acl/kitchen/positive.review",
    "data/MultiDomainSentiment/processed_acl/kitchen/unlabeled.review"
)


Starting processing...

Processing kitchen (1/3)
Reading file: data/MultiDomainSentiment/processed_acl/kitchen/negative.review

Processing kitchen (2/3)
Reading file: data/MultiDomainSentiment/processed_acl/kitchen/positive.review

Processing kitchen (3/3)
Reading file: data/MultiDomainSentiment/processed_acl/kitchen/unlabeled.review


In [10]:

# Books
df_books, df_books_testing = process_category(
    "books",
    "data/MultiDomainSentiment/processed_acl/books/negative.review",
    "data/MultiDomainSentiment/processed_acl/books/positive.review",
    "data/MultiDomainSentiment/processed_acl/books/unlabeled.review"
)




Processing books (1/3)
Reading file: data/MultiDomainSentiment/processed_acl/books/negative.review

Processing books (2/3)
Reading file: data/MultiDomainSentiment/processed_acl/books/positive.review

Processing books (3/3)
Reading file: data/MultiDomainSentiment/processed_acl/books/unlabeled.review


In [11]:
# Electronics
df_electronics, df_electronics_testing = process_category(
    "electronics",
    "data/MultiDomainSentiment/processed_acl/electronics/negative.review",
    "data/MultiDomainSentiment/processed_acl/electronics/positive.review",
    "data/MultiDomainSentiment/processed_acl/electronics/unlabeled.review"
)




Processing electronics (1/3)
Reading file: data/MultiDomainSentiment/processed_acl/electronics/negative.review

Processing electronics (2/3)
Reading file: data/MultiDomainSentiment/processed_acl/electronics/positive.review

Processing electronics (3/3)
Reading file: data/MultiDomainSentiment/processed_acl/electronics/unlabeled.review


In [12]:
# DVD
df_dvd, df_dvd_testing = process_category(
    "dvd",
    "data/MultiDomainSentiment/processed_acl/dvd/negative.review",
    "data/MultiDomainSentiment/processed_acl/dvd/positive.review",
    "data/MultiDomainSentiment/processed_acl/dvd/unlabeled.review"
)




Processing dvd (1/3)
Reading file: data/MultiDomainSentiment/processed_acl/dvd/negative.reviewte

Processing dvd (2/3)
Reading file: data/MultiDomainSentiment/processed_acl/dvd/positive.reviewte

Processing dvd (3/3)
Reading file: data/MultiDomainSentiment/processed_acl/dvd/unlabeled.reviewte


In [13]:
# Unir datasets de entrenamiento
df_training_categories = pd.concat([df_kitchen, df_books, df_electronics, df_dvd],ignore_index=True)

# Unir datasets de testing
df_testing = pd.concat([df_kitchen_testing, df_books_testing, df_electronics_testing, df_dvd_testing],ignore_index=True)

print("\nProcessing completed.")


Processing completed.


In [14]:
df_training_categories.to_csv("training_data_categories.csv")

In [15]:
df_testing.to_csv("testing_data_categories.csv")

# Preprocess

In [16]:
# Example data loading
df_train = pd.read_csv('training_data_categories.csv')

# Function to flatten the dictionary into a single string for vectorization
df_train['terms_str'] = df_train['terms'].apply(lambda x: ' '.join([f"{k} " * v for k, v in eval(x).items()]))
df_train['terms_str'] = df_train['terms_str'].apply(lambda x: processor_.preprocessing_pipeline_sentiments(x))


In [17]:
df_train

,Unnamed: 0,terms,label,category,terms_str
0,0,"{'right_after': 1, 'guess': 1, 'dog': 1, 'well...",negative,kitchen,right after guess dog well well fill to work i...
1,1,"{'manufacturer_suggested': 1, 'your': 2, 'manu...",negative,kitchen,manufacturer suggested manufacturer of the buy...
2,2,"{'time_it': 1, 'i_can': 1, 'my_cocker': 1, 'ma...",negative,kitchen,time it i can my cocker maybe it low depends c...
3,3,"{'save_your': 1, 'cheaply': 1, 'i_sent': 1, 's...",negative,kitchen,save your cheaply i sent save loose that right...
4,4,"{'acted': 1, 'sound_interesting': 1, 'they_are...",negative,kitchen,acted sound interesting they are bark all a co...
...,...,...,...,...,...
7995,7995,"{'z': 10, 'only': 1, 'course_of': 1, 'no': 5, ...",positive,dvd,z z z z z z z z z z course of help plenty like...
7996,7996,"{'well': 1, 'i': 1, 'interesting_as': 1, 'raid...",positive,dvd,well interesting as raiders liked this oakland...
7997,7997,"{'this_movie': 1, 'is_very': 1, 'enjoys_a': 1,...",positive,dvd,this movie is very enjoys a yet very very enjo...
7998,7998,"{'episodes_ommitted': 1, 'show': 2, ""gareth's""...",positive,dvd,episodes ommitted show show gareth america tel...


# Vectorizers

In [18]:
texto_procesado = df_train['terms_str'].tolist()
texto_procesado

['right after guess dog well well fill to work it seems keep other small work  num working very care barks cairn small dog time fairly sure why is supposed emits and barks just looks either does such a for my either away emits sure it either theyre getting to care the spray care if does not a persistent getting sprayed my other sprays as were not looks sprays if they are just is not my yappy too stubborn doesnt work looks away dog who barker barking we fill fill it not sure work work work sprayed i longit does also seems also does nt small cairn from sprayed it also yappy persistent barker not such persistent terriers isnt working longit just too num  of time and we from where right work well very longit well for supposed to when we terriers are after but keep my num getting stubborn yappy cairn the time as many stubborn to supposed where the well right fairly well nt nt work fairly many sprays why when barker terriers does work not have from barking away from many who is working he ju

countVectorizer

In [19]:
df_books = df_train[df_train['category'] == 'books']
df_electronics = df_train[df_train['category'] == 'electronics']
df_dvd = df_train[df_train['category'] == 'dvd']
df_kitchen = df_train[df_train['category'] == 'kitchen']

In [20]:
vectorizer = CountVectorizer(max_features=5000, stop_words=stopwords.words('english'))

In [21]:
def process_category(df, vectorizer):
    # Vectorizar los términos
    X_tf = vectorizer.fit_transform(df['terms_str']).toarray()
    # Obtener las etiquetas
    y = df['label'].values
    return X_tf, y

In [22]:

# Procesar las categorías
X_tf_books, y_books = process_category(df_books, vectorizer)
X_tf_electronics, y_electronics = process_category(df_electronics, vectorizer)
X_tf_dvd, y_dvd = process_category(df_dvd, vectorizer)
X_tf_kitchen, y_kitchen = process_category(df_kitchen, vectorizer)


In [23]:
def train_and_evaluate_with_validation(X, y, category_name, vectorizer):
    # Primera división: 70% (entrenamiento + validación) y 30% (prueba)
    X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    # Segunda división: del 70% restante, tomar 60% para entrenamiento y 10% para validación
    # 0.14285 es aproximadamente 10% / 70%, que nos da el 10% del conjunto de datos original
    X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1/0.7, random_state=0)
    
    # Entrenar Naive Bayes
    nb = MultinomialNB()
    nb.fit(X_train, y_train)
    y_pred_nb = nb.predict(X_test)
    
    print(f"################### Starting with category: {category_name} ###################")
    
    # Evaluar Naive Bayes
    print("\n")
    print(f"Resultados para Naive Bayes - {category_name}")
    evaluate_model(y_test, y_pred_nb, "Naive Bayes")
    
    # Entrenar Regresión Logística
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X_train, y_train)
    y_pred_lr = lr.predict(X_test)
    
    # Evaluar Regresión Logística
    print("\n")
    print(f"Resultados para Regresión Logística - {category_name}")
    evaluate_model(y_test, y_pred_lr, "Logistic Regression")
    
    # Evaluar en el conjunto de validación
    y_pred_val_nb = nb.predict(X_val)
    y_pred_val_lr = lr.predict(X_val)
    
    print("\n")
    print(f"Validación para Naive Bayes - {category_name}")
    evaluate_model(y_val, y_pred_val_nb, "Naive Bayes Validation")
    
    print("\n")
    print(f"Validación para Regresión Logística - {category_name}")
    evaluate_model(y_val, y_pred_val_lr, "Logistic Regression Validation")

    print("\n")
    # Extraer y mostrar las características más importantes en Regresión Logística
    print(f"\nCaracterísticas más importantes para Regresión Logística - {category_name}:")
    top_features, bottom_features = get_top_features_logistic_regression(lr, vectorizer, top_n=10)

    print("\n")
    
    print(f"################### Top 10 Features for {category_name} ###################")
    
    print("Top 10 características más importantes (asociadas con clase positiva):")
    for feature, coef in top_features:
        print(f"{feature}: {coef}")

    print("\n")
    print("\nTop 10 características menos importantes (asociadas con clase negativa):")
    for feature, coef in bottom_features:
        print(f"{feature}: {coef}")
    
    print("\n")

def evaluate_model(y_test, y_pred, model_name):
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', pos_label='positive')
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Evaluación de {model_name}:\n Precision: {precision}\n Recall: {recall}\n F1: {f1}\n Accuracy: {accuracy}\n")

def get_top_features_logistic_regression(lr_model, vectorizer, top_n=10):
    """
    Obtiene las características más importantes de un modelo de Regresión Logística.
    
    Args:
    lr_model: El modelo de regresión logística entrenado.
    vectorizer: El vectorizador utilizado (CountVectorizer o TfidfVectorizer).
    top_n: Número de características más importantes a extraer.
    
    Returns:
    Una lista de las características más importantes y sus coeficientes.
    """
    # Obtener los coeficientes del modelo
    coef = lr_model.coef_[0]
    
    # Obtener los nombres de las características desde el vectorizador
    feature_names = vectorizer.get_feature_names_out()
    
    # Ordenar los coeficientes por su valor absoluto para obtener las características más importantes
    sorted_idx = np.argsort(coef)[::-1]
    
    # Obtener las top_n características más importantes
    top_features = [(feature_names[i], coef[i]) for i in sorted_idx[:top_n]]
    bottom_features = [(feature_names[i], coef[i]) for i in sorted_idx[-top_n:]]
    
    return top_features, bottom_features


In [24]:


# Libros
train_and_evaluate_with_validation(X_tf_books, y_books, 'Books', vectorizer)

# Electrónica
train_and_evaluate_with_validation(X_tf_electronics, y_electronics, 'Electronics', vectorizer)

# DVD
train_and_evaluate_with_validation(X_tf_dvd, y_dvd, 'DVD', vectorizer)

# Cocina
train_and_evaluate_with_validation(X_tf_kitchen, y_kitchen, 'Kitchen', vectorizer)


################### Starting with category: Books ###################


Resultados para Naive Bayes - Books
Evaluación de Naive Bayes:
 Precision: 0.7542662116040956
 Recall: 0.7568493150684932
 F1: 0.7555555555555555
 Accuracy: 0.7616666666666667



Resultados para Regresión Logística - Books
Evaluación de Logistic Regression:
 Precision: 0.7559322033898305
 Recall: 0.7636986301369864
 F1: 0.7597955706984668
 Accuracy: 0.765



Validación para Naive Bayes - Books
Evaluación de Naive Bayes Validation:
 Precision: 0.75
 Recall: 0.7419354838709677
 F1: 0.745945945945946
 Accuracy: 0.7661691542288557



Validación para Regresión Logística - Books
Evaluación de Logistic Regression Validation:
 Precision: 0.7731958762886598
 Recall: 0.8064516129032258
 F1: 0.7894736842105263
 Accuracy: 0.8009950248756219




Características más importantes para Regresión Logística - Books:


################### Top 10 Features for Books ###################
Top 10 características más importantes (asociadas c

tfIdf

In [25]:
tfidf = TfidfVectorizer(max_features=5000, stop_words=stopwords.words('english'))


In [26]:
X_tf_books, y_books = process_category(df_books, tfidf)
X_tf_electronics, y_electronics = process_category(df_electronics, tfidf)
X_tf_dvd, y_dvd = process_category(df_dvd, tfidf)
X_tf_kitchen, y_kitchen = process_category(df_kitchen, tfidf)

In [27]:
# Libros
train_and_evaluate_with_validation(X_tf_books, y_books, 'Books', tfidf)

# Electrónica
train_and_evaluate_with_validation(X_tf_electronics, y_electronics, 'Electronics', tfidf)

# DVD
train_and_evaluate_with_validation(X_tf_dvd, y_dvd, 'DVD', tfidf)

# Cocina
train_and_evaluate_with_validation(X_tf_kitchen, y_kitchen, 'Kitchen')


################### Starting with category: Books ###################


Resultados para Naive Bayes - Books
Evaluación de Naive Bayes:
 Precision: 0.7733333333333333
 Recall: 0.7945205479452054
 F1: 0.7837837837837838
 Accuracy: 0.7866666666666666



Resultados para Regresión Logística - Books
Evaluación de Logistic Regression:
 Precision: 0.7523510971786834
 Recall: 0.821917808219178
 F1: 0.7855973813420621
 Accuracy: 0.7816666666666666



Validación para Naive Bayes - Books
Evaluación de Naive Bayes Validation:
 Precision: 0.7731958762886598
 Recall: 0.8064516129032258
 F1: 0.7894736842105263
 Accuracy: 0.8009950248756219



Validación para Regresión Logística - Books
Evaluación de Logistic Regression Validation:
 Precision: 0.7623762376237624
 Recall: 0.8279569892473119
 F1: 0.7938144329896907
 Accuracy: 0.8009950248756219




Características más importantes para Regresión Logística - Books:


################### Top 10 Features for Books ###################
Top 10 características m

TypeError: train_and_evaluate_with_validation() missing 1 required positional argument: 'vectorizer'